In [ ]:
!pip install --no-deps '/kaggle/input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '/kaggle/input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
!pip install ensemble-boxes

In [ ]:
import sys
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")
sys.path.insert(0, "../input/weightedboxesfusion")

import os
from ensemble_boxes import *
import torch
import random
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from tqdm import tqdm
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet.efficientdet import HeadNet
from sklearn.model_selection import StratifiedKFold

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [ ]:
def load_net(checkpoint_path):
    
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size=512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()


def load_net7(checkpoint_path):
    
    config = get_efficientdet_config('tf_efficientdet_d7')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size=512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

models = [

    load_net('../input/bestcheckpoint115epochs/memphisbestfold2.bin'),
    load_net('../input/bestcheckpoint115epochs/memphisbestfold2mixup1.bin'),
    load_net('../input/bestcheckpoint115epochs/memphisbestfold2mixup2.bin'),
    load_net('../input/bestcheckpoint115epochs/memphisbestfold2mixup3.bin'),
    
]



load_net('../input/pseudo/michaelbestfold3allpsuedo.bin'),
load_net('../input/pseudo/michaelbestfold4pseudo.bin'),
load_net('../input/pseudo/michaelbestfold0pseudo.bin'),
load_net('../input/pseudo/omkarpsuedomodel.bin'),

    load_net('../input/fold0andmixup/michaelbestfold0.bin')
    load_net('../input/fold0andmixup/michaelbestfold0mixup1.bin'),
    load_net('../input/fold0andmixup/michaelbestfold0mixup2.bin'),
    load_net('../input/fold0andmixup/michaelbestfold0mixup3.bin'),
    load_net('../input/bestcheckpoint115epochs/memphisbestfold2.bin'),
    load_net('../input/bestcheckpoint115epochs/memphisbestfold2mixup1.bin'),
    load_net('../input/bestcheckpoint115epochs/memphisbestfold2mixup2.bin'),
    load_net('../input/bestcheckpoint115epochs/memphisbestfold2mixup3.bin'),
    load_net('../input/epoch92031936/best-checkpoint-092epoch(0.32642-0.31936).bin'),
    load_net('../input/fold3andmixup/michaelbestfold3mixup1.bin'),
    load_net('../input/fold3andmixup/michaelbestfold3try1.bin'),
    load_net('../input/fold3andmixup/michaelbestfold3try2.bin'),
    load_net('../input/fold3andmixup/michaelbestfold3mixup2.bin'),
    load_net('../input/fold3andmixup/michaelbestfold3mixup3.bin'),
    load_net('../input/fold3andmixup/michaelbestfold3all.bin'),
    load_net('../input/fold4andmixup/michaelbestfold4mixup1.bin'),
    load_net('../input/fold4andmixup/michaelbestfold4mixup2.bin'),
    load_net('../input/fold4andmixup/michaelbestfold4mixup3.bin')
    

In [ ]:
DATA_ROOT_PATH = '../input/global-wheat-detection/test'

class TestDatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
    )

dataset = TestDatasetRetriever(
    image_ids=np.array([path.split('/')[-1][:-4] for path in glob(f'{DATA_ROOT_PATH}/*.jpg')]),
    transforms=get_valid_transforms()
)

def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

def make_predictions(
    images, 
    score_threshold=0.1,
):
    images = torch.stack(images).cuda().float()
    predictions = []
    for fold_number, net in enumerate(models):
        with torch.no_grad():
            det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
            result = []
            for i in range(images.shape[0]):
                boxes = det[i].detach().cpu().numpy()[:,:4]    
                scores = det[i].detach().cpu().numpy()[:,4]
                indexes = np.where(scores > score_threshold)[0]
                boxes = boxes[indexes]
                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                result.append({
                    'boxes': boxes[indexes],
                    'scores': scores[indexes],
                })
            predictions.append(result)
    return predictions


def run_wbf(predictions, image_index, image_size=512, iou_thr=0.44, skip_box_thr=0.43, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels


def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)


results = []

for images, image_ids in data_loader:
    predictions = make_predictions(images)
    for i, image in enumerate(images):
        boxes, scores, labels = run_wbf(predictions, image_index=i)
        boxes = (boxes*2).astype(np.int32).clip(min=0, max=1023)
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }
        results.append(result)
        
        
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)

In [ ]:
results

In [ ]:
test_df